# Limpieza LIGIE
## Fases de Tratado
### Fase 0: Configuración General

Antes de iniciar cualquier lógica, es fundamental preparar el entorno de ejecución instalando las dependencias necesarias y comprendiendo las herramientas que manipularán el archivo PDF y las estructuras de datos.

Instalación de dependencias: Para asegurar que el script funcione correctamente, se deben instalar las librerías externas que no vienen incluidas por defecto en Python. Ejecuta el siguiente comando en tu terminal o en una celda de tu entorno de desarrollo:

- pip install pdfplumber pandas numpy xlsxwriter

Librerías:

- pdfplumber: Es el motor principal de extracción. Se elige sobre otras opciones (como PyPDF2) porque permite estrategias visuales (detección de líneas de tablas) y lectura de texto posicional precisa.

- pandas (pd): Utilizada para crear los DataFrames. Es vital para manipular filas, columnas, rellenar datos faltantes (NaN) y transformar texto masivamente.

- numpy (np): Se usa auxiliarmente para definir valores nulos (np.nan) que pandas pueda reconocer y tratar.

- re: Librería de Expresiones Regulares. Es el "cerebro" que identifica patrones de texto complejos (ej. detectar si una línea empieza con "Sección IV" o "Capítulo 85").

- os: Utilidad de sistema operativo para manejo de rutas de archivos.

Variables Globales:

- os.chdir(''): Define el directorio de trabajo.

- PDF_PATH: Define la ruta del archivo origen.

- CODIGO_STOP: El script dejará de leer el PDF cuando encuentre este código específico (9807.00.01), optimizando tiempo al evitar leer anexos irrelevantes al final del documento.

In [ ]:
#pip install pdfplumber pandas numpy xlsxwriter

import pdfplumber
import pandas as pd
import numpy as np
import re

PDF_PATH = "../data/raw/LIGIE-UNIFICADA-LIGIE_20250728-20250728.pdf"
CODIGO_STOP = "9807.00.01"

REGEX_SECCION = re.compile(r"^(?:Sección|SECCIÓN|Seccion)\s+([IVXLCDM]+)", re.IGNORECASE) 
REGEX_CAPITULO = re.compile(r"^(?:Capítulo|CAPÍTULO|Capitulo)\s+(\d+)", re.IGNORECASE)

TEXTOS_BASURA = [
    "Calle Pachuca", "www.gob.mx", "Tel:", "(55) 5729", 
    "Economía", "Secretaría de Economía", 
    "Dirección General", "Facilitación", "Comercial y de Comercio Exterior",
    "PISTON MOLLICA", "Versión unificada", "Dudas y/o comentarios",
    "nueva.ligie", "Hoja", "Página", "The following table", 
    "Índice de abreviaturas", "Documento referencial", 
    "Ley de los Impuestos", "7ma Enmienda", "Acuerdo por el que"
]

### Fase 0.5: Definición de Funciones

En esta sección centralizamos toda la lógica del script. Esto permite mantener las fases posteriores limpias, conteniendo únicamente la ejecución del código. Las funciones se clasifican según su naturaleza:

**1. Funciones Auxiliares (`_nombre`)**
Herramientas de soporte para tareas repetitivas de limpieza y validación.

* **`_es_linea_util`**: Filtro de calidad. Recibe una línea de texto y decide si es información válida o "basura" (números de página, direcciones, encabezados repetitivos).
* **`_limpiar_descripcion`**: Formateador de texto. Elimina los caracteres de relleno del índice (como `....`) y números de página para dejar limpia la descripción de un Capítulo.
* **`_es_fila_basura`**: Filtro de tablas. Detecta si una fila extraída de una tabla es en realidad un encabezado repetido (ej. contiene "TASA" o "IMPUESTO").
* **`_get_seccion_romana`**: Buscador simple. Dado un código de Capítulo (ej. "01"), busca en el mapa generado a qué Sección Romana pertenece (ej. "I").
* **`_concatenar_descripciones`**: Constructor de texto. Toma las partes jerárquicas (Sección, Capítulo, Partida, Fracción) y las une en una sola oración fluida.

**2. Funciones Críticas (`__nombre__`)**
Lógica estructural compleja o delicada que no debe modificarse a la ligera.

* **`__buscar_descripcion_real__`**: Motor de búsqueda *Lookahead*. Es el corazón de la lectura del índice. Si encuentra un título (ej. "Capítulo 85") sin descripción, escanea las líneas siguientes de manera inteligente hasta encontrar el texto correcto, saltándose la basura.
* **`__escribir_hoja__`**: Motor de exportación Excel. Controla la librería `xlsxwriter` para crear pestañas, aplicar negritas, bordes y, lo más importante, calcular el ajuste de texto (Wrap Text) y anchos de columna para que el reporte sea legible.

**3. Funciones Principales (`nombre`)**
Ejecutan procesos completos.

* **`generar_catalogos_final`**: Coordina la lectura masiva de las primeras páginas del PDF, invoca a las funciones regex y de búsqueda (`__buscar_descripcion_real__`) y retorna los diccionarios maestros de Secciones y Capítulos listos para usar.

In [ ]:
# --- FUNCIONES AUXILIARES ---

def _es_linea_util(txt):
    if not txt: return False
    txt_lower = txt.lower()
    if any(basura.lower() in txt_lower for basura in TEXTOS_BASURA): return False
    if re.match(r'^[\d\.\s]+$', txt): return False
    if len(txt) < 3: return False
    return True


def _limpiar_descripcion(txt):
    txt = re.split(r'\.{2,}', txt)[0]
    txt = re.sub(r'\s+\d+$', '', txt)
    txt = txt.lstrip('.- ').strip()
    return txt


def _es_fila_basura(row, palabras_prohibidas):
    texto_fila = " ".join([str(x) for x in row if x is not None]).upper()
    return any(palabra in texto_fila for palabra in palabras_prohibidas)


def _get_seccion_romana(cap_code, mapa):
    return mapa.get(cap_code, "ND")


def _concatenar_descripciones(row):
    items = [
        row.get('Txt_Seccion_Nombre', ''),
        row.get('Txt_Capitulo', ''),
        row.get('Txt_Partida', ''),
        row.get('Txt_Desdoblamiento', ''),
        row.get('Txt_Subpartida', ''),
        row.get('DESCRIPCION', '')
    ]
    return " ".join([str(x).strip() for x in items if str(x).strip() != '' and str(x).strip() != 'nan'])


# --- FUNCIONES CRÍTICAS ---

def __buscar_descripcion_real__(start_index, match_obj, all_lines):
    total_lines = len(all_lines)
    
    # A. Intentar en la misma línea
    parte_match = match_obj.group(0)
    resto_linea = all_lines[start_index][len(parte_match):].strip()
    resto_linea = re.split(r'\.{2,}', resto_linea)[0].strip()
    
    if len(resto_linea) > 2 and not re.match(r'^[\d\.]+$', resto_linea):
        return _limpiar_descripcion(resto_linea)
    
    # B. Buscar hacia abajo (Lookahead)
    for offset in range(1, 15):
        if start_index + offset >= total_lines: break
        next_line = all_lines[start_index + offset].strip()
        
        if REGEX_SECCION.match(next_line) or REGEX_CAPITULO.match(next_line):
            return None 
        
        if _es_linea_util(next_line):
            return _limpiar_descripcion(next_line)
    return None


def __escribir_hoja__(dataframe, nombre_hoja, writer, workbook):
    # Definición de formatos
    fmt_header = workbook.add_format({'bold': True, 'border': 1, 'bg_color': '#D9E1F2', 'valign': 'top', 'align': 'center', 'font_name': 'Arial', 'font_size': 10})
    fmt_normal = workbook.add_format({'border': 1, 'valign': 'top', 'font_name': 'Arial', 'font_size': 9})
    fmt_wrap = workbook.add_format({'border': 1, 'valign': 'top', 'text_wrap': True, 'font_name': 'Arial', 'font_size': 9})

    worksheet = workbook.add_worksheet(nombre_hoja)
    writer.sheets[nombre_hoja] = worksheet
    
    headers = dataframe.columns.tolist()
    for col_idx, header in enumerate(headers):
        worksheet.write(0, col_idx, header, fmt_header)
        
    for row_idx, row in enumerate(dataframe.itertuples(index=False), start=1):
        for col_idx, value in enumerate(row):
            header_name = headers[col_idx]
            columnas_largas = ["Nombre", "DESCRIPCION", "Descripción", "ACOTACION", "Concatenada"]
            es_texto_largo = any(x in header_name for x in columnas_largas)
            formato = fmt_wrap if es_texto_largo else fmt_normal
            if pd.isna(value): value = ""
            worksheet.write(row_idx, col_idx, value, formato)
    
    # Ajuste de anchos
    for idx, col_name in enumerate(headers):
        if col_name in ['Capítulo', 'Partida', 'Desdob.', 'Desdoblamiento', 'Subpartida', 'Fracción', 'Sección']:
             worksheet.set_column(idx, idx, 8) 
        elif any(x in col_name for x in ["Nombre", "DESCRIPCION", "Descripción", "ACOTACION", "Concatenada"]):
            worksheet.set_column(idx, idx, 50) 
        else:
            worksheet.set_column(idx, idx, 12)


# --- FUNCIONES PRINCIPALES ---

def generar_catalogos_final(pdf_path, paginas_a_escanear=25):
    print(f"--- FASE 1: Generando catálogos (Escanear primeras {paginas_a_escanear} págs) ---")
    
    dict_capitulos = {}
    dict_secciones = {}
    mapa_cap_sec = {} 
    
    # 1. Carga masiva de líneas
    all_lines = []
    try:
        with pdfplumber.open(pdf_path) as pdf:
            limit = min(paginas_a_escanear, len(pdf.pages))
            for i in range(limit):
                page = pdf.pages[i]
                text = page.extract_text()
                if text:
                    lines = [l.strip() for l in text.split('\n') if l.strip()]
                    all_lines.extend(lines)
    except Exception as e:
        print(f"Error crítico leyendo PDF para catálogos: {e}")
        return {}, {}, {}

    # 2. Procesamiento usando funciones externas
    seccion_actual_romana = "ND"

    for idx, line in enumerate(all_lines):
        match_sec = REGEX_SECCION.match(line)
        match_cap = REGEX_CAPITULO.match(line)

        if match_sec:
            num = match_sec.group(1).strip()
            seccion_actual_romana = num
            # Llamada a función crítica
            desc = __buscar_descripcion_real__(idx, match_sec, all_lines)
            if desc:
                dict_secciones[num] = desc.upper()
        
        elif match_cap:
            num_raw = match_cap.group(1).strip()
            key = f"{int(num_raw):02d}"
            mapa_cap_sec[key] = seccion_actual_romana
            # Llamada a función crítica
            desc = __buscar_descripcion_real__(idx, match_cap, all_lines)
            if desc:
                dict_capitulos[key] = desc
            else:
                dict_capitulos[key] = "DESCRIPCIÓN NO ENCONTRADA"

    return dict_capitulos, dict_secciones, mapa_cap_sec

### Fase 1: Generación de Catálogos

Esta fase ejecuta la lectura del índice inicial. Gracias a la definición previa en la Fase 0.5, el código aquí es puramente ejecutable.

- Se invoca a `generar_catalogos_final` con la ruta del PDF.
- Se reciben los diccionarios limpios.

In [3]:
CAPITULOS_NOMBRES, SECCIONES_NOMBRES, MAPA_CAPITULO_SECCION = generar_catalogos_final(PDF_PATH)
print(f"Catálogos listos: {len(CAPITULOS_NOMBRES)} Capítulos detectados.")

--- FASE 1: Generando catálogos (Escanear primeras 25 págs) ---
Catálogos listos: 98 Capítulos detectados.


### Fase 2: Extracción de Tablas

Una vez que tenemos los catálogos, pasamos a extraer la información del resto del documento.

- **Estrategia**: Se iteran las páginas buscando la palabra "CÓDIGO" para recortar encabezados.
- **Limpieza**: Se utiliza la función auxiliar `_es_fila_basura` para limpiar los datos crudos.
- **Parada**: Se busca el `CODIGO_STOP`.

In [4]:
all_data = []
COLUMNAS_ESPERADAS = ["CODIGO", "NICO", "DESCRIPCION", "UNIDAD", "IMP_IMP", "IMP_EXP", "ACOTACION"]
PALABRAS_PROHIBIDAS = ["SUBP", "IMPUESTO", "EXP.", "IMP.", "TASA", "CUOTA", "TRATADO", "ARANCEL"]
stop_process_flag = False

print(f"\n--- FASE 2: Extracción de tablas de datos ---")

try:
    with pdfplumber.open(PDF_PATH) as pdf:
        total = len(pdf.pages)
        start_page = 0 
        
        for i in range(start_page, total):
            if stop_process_flag: break
            page = pdf.pages[i]

            # Buscar encabezado de tabla
            busqueda = page.search("CÓDIGO")
            crop_y = 0
            found_header = False

            if busqueda:
                crop_y = busqueda[0]["bottom"] + 2
                found_header = True

            if found_header:
                try:
                    cropped_page = page.crop((0, crop_y, page.width, page.height))
                    table_settings = {"vertical_strategy": "lines", "horizontal_strategy": "lines", "snap_tolerance": 4}
                    table = cropped_page.extract_table(table_settings)

                    if table:
                        df = pd.DataFrame(table)
                        df = df.dropna(how='all')

                        # Limpiar filas basura repetitivas usando función auxiliar
                        while not df.empty and _es_fila_basura(df.iloc[0], PALABRAS_PROHIBIDAS):
                            df = df.iloc[1:]
                            df.reset_index(drop=True, inplace=True)

                        if df.shape[1] >= 7:
                            conteos = df.count()
                            indices_top = conteos.nlargest(7).index.sort_values()
                            df = df[indices_top]
                            df.columns = COLUMNAS_ESPERADAS

                            mask_basura = (df['CODIGO'].isna() | (df['CODIGO'] == '')) & \
                                          (df['DESCRIPCION'].str.contains("Página", case=False, na=False))
                            df = df[~mask_basura]

                            codigos_str = df['CODIGO'].astype(str).str.strip()
                            if CODIGO_STOP in codigos_str.values:
                                print(f"¡OBJETIVO ENCONTRADO! Código {CODIGO_STOP} detectado en pág {i+1}.")
                                idx_stop = df[codigos_str == CODIGO_STOP].index[0]
                                df = df.iloc[:idx_stop + 1]
                                stop_process_flag = True

                            all_data.append(df)
                except Exception: pass

            if i % 50 == 0: print(f"Procesando página {i+1}/{total}...")

except Exception as e:
    print(f"Error abriendo PDF en Fase 2: {e}")


--- FASE 2: Extracción de tablas de datos ---
Procesando página 1/1315...
Procesando página 51/1315...
Procesando página 101/1315...
Procesando página 151/1315...
Procesando página 201/1315...
Procesando página 251/1315...
Procesando página 301/1315...
Procesando página 351/1315...
Procesando página 401/1315...
Procesando página 451/1315...
Procesando página 501/1315...
Procesando página 551/1315...
Procesando página 601/1315...
Procesando página 651/1315...
Procesando página 701/1315...
Procesando página 751/1315...
Procesando página 801/1315...
Procesando página 851/1315...
Procesando página 901/1315...
Procesando página 951/1315...
Procesando página 1001/1315...
Procesando página 1051/1315...
Procesando página 1101/1315...
Procesando página 1151/1315...
Procesando página 1201/1315...
¡OBJETIVO ENCONTRADO! Código 9807.00.01 detectado en pág 1226.


### Fase 3: Procesamiento y Lógica Jerárquica

Aquí ocurre la transformación de datos crudos a información estructurada.

- **Relleno**: Uso de `bfill()`.
- **Mapeo**: Uso de `_get_seccion_romana`.
- **Concatenación**: Uso de `_concatenar_descripciones`.

In [ ]:
if all_data:
    print("\n--- FASE 3: Procesamiento y Exportación ---")
    final_df = pd.concat(all_data, ignore_index=True)
    final_df = final_df.replace(r'\n', ' ', regex=True)

    # 1. Relleno Jerárquico
    print("Aplicando relleno de códigos...")
    final_df['NICO'] = final_df['NICO'].fillna('').astype(str).str.strip().replace(['--', '-'], '')
    final_df['CODIGO'] = final_df['CODIGO'].replace(r'^\s*$', np.nan, regex=True)
    
    siguientes_codigos = final_df['CODIGO'].bfill()
    cond_codigo_vacio = final_df['CODIGO'].isna()
    cond_nico_vacio = final_df['NICO'] == ''
    mascara = cond_codigo_vacio & cond_nico_vacio
    
    if not siguientes_codigos[mascara].empty:
        final_df.loc[mascara, 'CODIGO'] = siguientes_codigos[mascara].astype(str).str[:-1]

    final_df = final_df.fillna('')

    # 2. Generación de Diccionarios
    print("Extrayendo descripciones jerárquicas...")
    panel_base = final_df.copy()
    panel_base['CODIGO_LIMPIO'] = panel_base['CODIGO'].astype(str).str.replace('.', '', regex=False).str.strip()

    dict_partidas = {}       # 4 dígitos
    dict_desdoblamiento = {} # 5 dígitos
    dict_subpartidas = {}    # 6 dígitos

    for row in panel_base.itertuples():
        code = getattr(row, 'CODIGO_LIMPIO', '')
        desc = getattr(row, 'DESCRIPCION', '')
        
        if len(code) == 4:
            dict_partidas[code] = desc
        elif len(code) == 5:
            dict_desdoblamiento[code] = desc
        elif len(code) == 6:
            dict_subpartidas[code] = desc

    # 3. Filtrado Final
    panel_df = panel_base[
        (panel_base['CODIGO_LIMPIO'].str.len() == 8) &
        (panel_base['IMP_IMP'].str.strip() != '')
    ].copy()
    
    panel_df = panel_df.drop_duplicates(subset=['CODIGO', 'NICO'], keep='first')

    # 4. Desglose de Dígitos
    print("Desglosando dígitos...")
    code_full = panel_df['CODIGO_LIMPIO']
    
    panel_df['Show_Capitulo']       = code_full.str[:2]
    panel_df['Show_Partida']        = code_full.str[2:4]
    panel_df['Show_Desdoblamiento'] = code_full.str[4:5]
    panel_df['Show_Subpartida']     = code_full.str[5:6]
    panel_df['Show_Fraccion']       = code_full.str[6:8]

    # 5. Mapeo de Textos
    panel_df['Txt_Seccion_Romana'] = code_full.str[:2].apply(lambda x: _get_seccion_romana(x, MAPA_CAPITULO_SECCION))
    
    panel_df['Txt_Seccion_Nombre'] = panel_df['Txt_Seccion_Romana'].map(SECCIONES_NOMBRES)
    panel_df['Txt_Capitulo']       = code_full.str[:2].map(CAPITULOS_NOMBRES)
    panel_df['Txt_Partida']        = code_full.str[:4].map(dict_partidas)
    panel_df['Txt_Desdoblamiento'] = code_full.str[:5].map(dict_desdoblamiento)
    panel_df['Txt_Subpartida']     = code_full.str[:6].map(dict_subpartidas)

    # 6. Homogenización de Formatos de Texto
    print("Formateando textos (Mayúsculas y Puntos)...")
    
    panel_df['Txt_Seccion_Nombre'] = panel_df['Txt_Seccion_Nombre'].fillna('').astype(str).str.lower().str.capitalize()
    panel_df['Txt_Seccion_Nombre'] = panel_df['Txt_Seccion_Nombre'].str.rstrip('.') + '.'
    panel_df['Txt_Capitulo'] = panel_df['Txt_Capitulo'].fillna('').astype(str).str.rstrip('.') + '.'

    panel_df['Codigo_Completo'] = panel_df['CODIGO']

    # 7. Concatenación usando función auxiliar externa
    print("Generando concatenación limpia...")
    panel_df['Texto_Concatenado'] = panel_df.apply(_concatenar_descripciones, axis=1)

    # --- DEFINICIÓN DE ESTRUCTURAS DE SALIDA ---
    cols_reducido = ['Codigo_Completo', 'Txt_Seccion_Romana', 'Show_Capitulo', 'Show_Partida', 'Show_Desdoblamiento', 'Show_Subpartida', 'Show_Fraccion', 'UNIDAD', 'IMP_IMP', 'IMP_EXP']
    df_reducido = panel_df[cols_reducido].copy()
    df_reducido.columns = ['Código Completo', 'Sección', 'Capítulo', 'Partida', 'Desdoblamiento', 'Subpartida', 'Fracción', 'Unidad', 'Imp. Imp.', 'Imp. Exp.']

    cols_textual = ['Codigo_Completo', 'Txt_Seccion_Nombre', 'Txt_Capitulo', 'Txt_Partida', 'Txt_Desdoblamiento', 'Txt_Subpartida', 'DESCRIPCION', 'UNIDAD', 'IMP_IMP', 'IMP_EXP']
    df_textual = panel_df[cols_textual].copy()
    df_textual.columns = ['Código Completo', 'Nombre Sección', 'Nombre Capítulo', 'Nombre Partida', 'Nombre Desdoblamiento', 'Nombre Subpartida', 'Descripción Fracción', 'Unidad', 'Imp. Imp.', 'Imp. Exp.']

    cols_extendido = ['Codigo_Completo', 'Txt_Seccion_Romana', 'Txt_Seccion_Nombre', 'Show_Capitulo', 'Txt_Capitulo', 'Show_Partida', 'Txt_Partida', 'Show_Desdoblamiento', 'Txt_Desdoblamiento', 'Show_Subpartida', 'Txt_Subpartida', 'Show_Fraccion', 'DESCRIPCION', 'UNIDAD', 'IMP_IMP', 'IMP_EXP']
    df_extendido = panel_df[cols_extendido].copy()
    df_extendido.columns = ['Código Completo', 'Sección', 'Nombre Sección', 'Capítulo', 'Nombre Capítulo', 'Partida', 'Nombre Partida', 'Desdob.', 'Nombre Desdoblamiento', 'Subpartida', 'Nombre Subpartida', 'Fracción', 'Descripción Fracción', 'Unidad', 'Imp. Imp.', 'Imp. Exp.']

    cols_concat = ['Codigo_Completo', 'Texto_Concatenado', 'UNIDAD', 'IMP_IMP', 'IMP_EXP']
    df_concatenado = panel_df[cols_concat].copy()
    df_concatenado.columns = ['Código Completo', 'Descripción Completa Concatenada', 'Unidad', 'Imp. Imp.', 'Imp. Exp.']

    # --- EXPORTACIÓN A EXCEL ---
    nombre_salida = "LIGIE_Limpia.xlsx"
    print(f"Generando archivo Excel: {nombre_salida} ...")

    writer = pd.ExcelWriter(nombre_salida, engine='xlsxwriter')
    workbook = writer.book

    # Llamadas a función crítica de escritura
    __escribir_hoja__(final_df, "LIGIE", writer, workbook) 
    __escribir_hoja__(df_reducido, "Panel Reducido", writer, workbook)
    __escribir_hoja__(df_textual, "Panel Textual", writer, workbook)
    __escribir_hoja__(df_extendido, "Panel Extendido", writer, workbook)
    __escribir_hoja__(df_concatenado, "Concatenado", writer, workbook)

    writer.close()
    print("¡PROCESO FINALIZADO CON ÉXITO!")

else:
    print("No se encontraron datos para exportar. Verifica el archivo PDF.")


--- FASE 3: Procesamiento y Exportación ---
Aplicando relleno de códigos...
Extrayendo descripciones jerárquicas...
Desglosando dígitos...
Formateando textos (Mayúsculas y Puntos)...
Generando concatenación limpia...
Generando archivo Excel: LIGIE_Maestra_Unificada.xlsx ...
¡PROCESO FINALIZADO CON ÉXITO!
